# Implementation of redundant example removal using hill climbing

The core components for this problem (or any hill climbing):

1. Initial state: entire training set
2. Search operator: removing an example
3. Evaluation function: using 1-NN classifier to see if suggested label for all examples match original label
4. Final state: consistent subset

Each component will be implemented independently.

In [5]:
import pandas as pd
import numpy as np
import heapq

In [2]:
pies = [
    ['circle', 'thick', 'gray', 'thick', 'dark', 'pos'],
    ['circle', 'thick', 'white', 'thick', 'dark', 'pos'],
    ['triangle', 'thick', 'dark', 'thick', 'gray', 'pos'],
    ['circle', 'thin', 'white', 'thin', 'dark', 'pos'],
    ['square', 'thick', 'dark', 'thin', 'white', 'pos'],
    ['circle', 'thick', 'white', 'thin', 'dark', 'pos'],
    ['circle', 'thick', 'gray', 'thick', 'white', 'neg'],
    ['square', 'thick', 'white', 'thick', 'gray', 'neg'],
    ['triangle', 'thin', 'gray', 'thin', 'dark', 'neg'],
    ['circle', 'thick', 'dark', 'thick', 'white', 'neg'],
    ['square', 'thick', 'white', 'thick', 'dark', 'neg'],
    ['triangle', 'thick', 'white', 'thick', 'gray', 'neg'],
]

labelled_pies = {}
for p in pies:
    if p[-1] not in labelled_pies:
        labelled_pies[p[-1]] = []
    labelled_pies[p[-1]].append(p)

labelled_pies

{'pos': [['circle', 'thick', 'gray', 'thick', 'dark', 'pos'],
  ['circle', 'thick', 'white', 'thick', 'dark', 'pos'],
  ['triangle', 'thick', 'dark', 'thick', 'gray', 'pos'],
  ['circle', 'thin', 'white', 'thin', 'dark', 'pos'],
  ['square', 'thick', 'dark', 'thin', 'white', 'pos'],
  ['circle', 'thick', 'white', 'thin', 'dark', 'pos']],
 'neg': [['circle', 'thick', 'gray', 'thick', 'white', 'neg'],
  ['square', 'thick', 'white', 'thick', 'gray', 'neg'],
  ['triangle', 'thin', 'gray', 'thin', 'dark', 'neg'],
  ['circle', 'thick', 'dark', 'thick', 'white', 'neg'],
  ['square', 'thick', 'white', 'thick', 'dark', 'neg'],
  ['triangle', 'thick', 'white', 'thick', 'gray', 'neg']]}

In [11]:
# Computing the distance between two vectors
# In this case, we are just using the boolean condition of true/false of whether there is a match
# The more matches, the closer it is, so we should use 1/matches to represent that
def dist(x, y):
    total = 0
    for [a, b] in zip(x, y):
        if a == b:
            total += 1
    if total == 0:
        # If no matches at all, we just return 1 being the furthest possible distance
        return 1
    return 1/total

# Expects 2
dist(['circle', 'thick', 'gray', 'thick', 'dark'], ['circle', 'thin', 'black', 'thick', 'white'])

0.5

In [16]:
# implementation of k-NN classifier where we can specify k
# given examples, find the k nearest neighbours for vector x
# use the dist(x, y) function above
def knn(k, examples, x):
    h = []
    for example in examples:
        heapq.heappush(h, (-dist(x, example), example))
        if len(h) > k:
            heapq.heappop(h)
    neighbors = []
    while h:
        neighbors.append(heapq.heappop(h)[1])
    return neighbors[::-1]

knn(3, pies, ['circle', 'thick', 'gray', 'thick', 'white'])

[['circle', 'thick', 'gray', 'thick', 'white', 'neg'],
 ['circle', 'thick', 'gray', 'thick', 'dark', 'pos'],
 ['circle', 'thick', 'dark', 'thick', 'white', 'neg']]

In [17]:
# uses knn() above but now returns the label instead
def knn_classifier(k, examples, x):
    neighbors = knn(k, examples, x)
    freq = {}
    for n in neighbors:
        freq[n[-1]] = freq.get(n[-1], 0) + 1
    label = ''
    label_f = 0
    for l, f in freq.items():
        if f > label_f:
            label = l
            label_f = f
    return label

knn_classifier(3, pies, ['circle', 'thick', 'gray', 'thick', 'white'])

'neg'

In [20]:
# Evaluation function where 1-NN classifier is used to generate "potential" labels
# The core idea is for every example, we perform 1-NN on itself to find if the 1-NN classifier returns
#   the same label that it has originally
# So for every example, we should general the "match" score and prioritize using the highest match score
def evaluate(examples):
    matches = 0
    for example in examples:
        predicted_label = knn_classifier(1, examples, example)
        actual_label = example[-1]
        if predicted_label == actual_label:
            matches += 1
    return matches

# initially the 1-NN should be itself so no change
evaluate(pies)

12

In [23]:
# Now the implementation of the hill climbing
# For each possible set of training examples, we try deleting one of its examples and use the evaluation function
# if the result is a perfect match to the original size, then we can use this configuration
# if there is no configuration that works, that means we have found out consistent subset and cannot reduce it further
def find_consistent_subset(initial):
    frontier = [initial]
    consistent_subsets = []
    while frontier:
        next_frontier = []
        for examples in frontier:
            is_consistent_subset = True
            for i in range(len(examples)):
                removed = examples[:i] + examples[i+1:]
                score = evaluate(removed)
                if score == len(initial):
                    next_frontier.append(removed)
                    is_consistent_subset = False
            if is_consistent_subset:
                consistent_subsets.append(examples)
        frontier = next_frontier

    return consistent_subsets

find_consistent_subset(pies)

[[['circle', 'thick', 'gray', 'thick', 'dark', 'pos'],
  ['circle', 'thick', 'white', 'thick', 'dark', 'pos'],
  ['triangle', 'thick', 'dark', 'thick', 'gray', 'pos'],
  ['circle', 'thin', 'white', 'thin', 'dark', 'pos'],
  ['square', 'thick', 'dark', 'thin', 'white', 'pos'],
  ['circle', 'thick', 'white', 'thin', 'dark', 'pos'],
  ['circle', 'thick', 'gray', 'thick', 'white', 'neg'],
  ['square', 'thick', 'white', 'thick', 'gray', 'neg'],
  ['triangle', 'thin', 'gray', 'thin', 'dark', 'neg'],
  ['circle', 'thick', 'dark', 'thick', 'white', 'neg'],
  ['square', 'thick', 'white', 'thick', 'dark', 'neg'],
  ['triangle', 'thick', 'white', 'thick', 'gray', 'neg']]]